# Skapa en modell 
som ger underlag för att köpa eller sälja kryptovaluta

In [1]:
import pandas as pd
import numpy as np
# import binance
import momentum as mom

## Skapa train/test data från momentum.py

In [2]:
### Kör denna kod för att skapa en dataframe med train/test data ###
###               Gör interupt när tillräckligt finns            ###
# set timer
import time
 
start_time = time.time()
max_time = 60    # *10      # 10 minutes
mom.start(max_time)

duration = time.time() - start_time
# format duration to minutes and seconds
duration = divmod(duration, 60)
str_duration = str(duration[0]) + ' minutes and ' + str(int(duration[1])) + ' seconds'
print('Funktionen har kört',str_duration )

trading logic start
fibonacci_strategy start
start_a_new_fibonacci 2023-02-27 20:21:10.874000
stop_loss gick inte att fylla i
entry gick inte att fylla i
trading logic start
fibonacci_strategy start
return hold and in_position = False row = [1632.17 1608.0 1665.22 1616.0 475515.8268 nan nan 'hold' nan nan nan
 'hold']
(2, 16)
trading logic start
fibonacci_strategy start
return hold and in_position = False row = [1632.11 1608.0 1665.22 1616.37 475521.5834 nan nan 'hold' nan nan nan
 'hold' nan nan nan nan nan]
(3, 17)
trading logic start
fibonacci_strategy start
return hold and in_position = False row = [1632.11 1608.0 1665.22 1616.4 475522.2295 nan nan 'hold' nan nan nan
 'hold' nan nan nan nan nan]
(4, 17)
trading logic start
fibonacci_strategy start
return hold and in_position = False row = [1632.1 1608.0 1665.22 1616.39 475522.2385 1616.232 nan 'hold' nan nan nan
 'hold' nan nan nan nan nan]
(5, 17)
trading logic start
fibonacci_strategy start
return hold and in_position = False row

In [ ]:

print(mom.df.tail(10))
mom.df.to_csv('data.csv')
print(mom.df.shape)

In [ ]:
mom.df

In [ ]:
# use_features = mom.df.columns.tolist()

# # write to file
# with open('use_features.txt', 'w') as f:
#     for item in use_features:
#         f.write("%s " % item)
# use_features

## Skapa train/test data från get_historical_klines() i binance.Client

In [ ]:
import pandas as pd
import numpy as np
import binance

In [ ]:
client = binance.Client()
r = client.get_historical_klines('ETHBTC', client.KLINE_INTERVAL_1MINUTE, '1-Jan-2023', '20-Jan-2023')
df = pd.DataFrame(r,columns=['datum', 'open','low','high','close','volume','close_time','quote_asset_volume','number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore'])
df = df[['datum','open','high','low','close','volume']]
# OHLCV values (Open time, Open, High, Low, Close, Volume, Close time, Quote asset volume, 
# Number of trades, Taker buy base asset volume, Taker buy quote asset volume, Ignore)
df.datum = pd.to_datetime(df.datum, unit='ms')
df.set_index('datum', inplace=True)

# set all columns numeric
df = df.apply(pd.to_numeric, errors='coerce')

In [ ]:
print(df.info())
print(df.shape)
print(df.columns)

Prepare data for training and testing

In [ ]:
# read use_features.txt file
with open('use_features.txt', 'r') as f:
    use_features = f.read().split() 
    
# set all use_features to np.nan
for feature in use_features:
    if feature not in df.columns:
        df[feature] = np.nan

# df.pris = df.close
df['y_1'] = False
df['y_2'] = False
df['y_1'] = (df.close.shift(-1) > df.close).astype(int) # går close upp eller ner nästa minut?
df['y_2'] = (df.close.shift(-2) > df.close).astype(int) # går close upp eller ner om 2 minuter?
# df[use_features]

df

In [ ]:
df.y_2.value_counts()

In [ ]:
mom.df = df[use_features+['y_1','y_2']].copy()
mom.df.info()

In [ ]:
mom.trading_logic()

print(mom.df.info())
mom.df.tail(10)

In [ ]:
print(mom.df.m_strategy.value_counts())
print(mom.df.b_strategy.value_counts())
print(mom.df.f_strategy.value_counts())


RandomForrestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# select the 20% last rows for test
test = mom.df.iloc[-int(mom.df.shape[0]*0.2):].copy()
train = mom.df[~mom.df.index.isin(test.index)].copy()
test = test.dropna().copy()
train= train.dropna().copy()

# set y_test to 1 if y_1 > pris else 0
y_test = pd.Series(np.where(test['y_1'] > test['pris'], 1, 0))
y_train = pd.Series(np.where(train['y_1'] > train['pris'], 1, 0))
X_test = test[use_features].copy()
X_train= train[use_features].copy()

# replace 'hold' with 0, 'buy' with 1 and 'sell' with -1
X_train.replace({'hold':0.0, 'buy':1.0, 'sell':-1.0}, inplace=True)
X_test.replace({'hold':0.0, 'buy':1.0, 'sell':-1.0}, inplace=True)

In [ ]:
print(X_test.shape, y_test.shape)
print(X_train.shape, y_train.shape)

In [ ]:
print(np.array(X_test.pris))
print(np.array(y_test))

In [ ]:
# create model
rfc = RandomForestClassifier(n_estimators=1000, max_depth=12, random_state=0, n_jobs=-1)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
y_train_pred = rfc.predict(X_train)

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))


In [ ]:

print(confusion_matrix(y_train, y_train_pred))
print(accuracy_score(y_train, y_train_pred))
print(roc_auc_score(y_train, y_train_pred))

In [ ]:
y_train.value_counts()